In [30]:
import pandas as pd
import numpy as np

In [3]:
df=pd.read_csv(r'E:\DiamondPricePrediction\notebooks\data\gemstone.csv')

In [4]:
df.head()

,id,carat,cut,color,clarity,depth,table,x,y,z,price
0,0,1.52,Premium,F,VS2,62.2,58.0,7.27,7.33,4.55,13619
1,1,2.03,Very Good,J,SI2,62.0,58.0,8.06,8.12,5.05,13387
2,2,0.70,Ideal,G,VS1,61.2,57.0,5.69,5.73,3.50,2772
3,3,0.32,Ideal,G,VS1,61.6,56.0,4.38,4.41,2.71,666
4,4,1.70,Premium,G,VS2,62.6,59.0,7.65,7.61,4.77,14453


In [5]:
df.drop(columns=['id'],axis=1,inplace=True)

In [6]:
df.head()

,carat,cut,color,clarity,depth,table,x,y,z,price
0,1.52,Premium,F,VS2,62.2,58.0,7.27,7.33,4.55,13619
1,2.03,Very Good,J,SI2,62.0,58.0,8.06,8.12,5.05,13387
2,0.70,Ideal,G,VS1,61.2,57.0,5.69,5.73,3.50,2772
3,0.32,Ideal,G,VS1,61.6,56.0,4.38,4.41,2.71,666
4,1.70,Premium,G,VS2,62.6,59.0,7.65,7.61,4.77,14453


In [7]:
X=df.drop(columns=['price'],axis=1)

In [8]:
Y=df['price']

In [9]:
X

,carat,cut,color,clarity,depth,table,x,y,z
0,1.52,Premium,F,VS2,62.2,58.0,7.27,7.33,4.55
1,2.03,Very Good,J,SI2,62.0,58.0,8.06,8.12,5.05
2,0.70,Ideal,G,VS1,61.2,57.0,5.69,5.73,3.50
3,0.32,Ideal,G,VS1,61.6,56.0,4.38,4.41,2.71
4,1.70,Premium,G,VS2,62.6,59.0,7.65,7.61,4.77
...,...,...,...,...,...,...,...,...,...
193568,0.31,Ideal,D,VVS2,61.1,56.0,4.35,4.39,2.67
193569,0.70,Premium,G,VVS2,60.3,58.0,5.75,5.77,3.47
193570,0.73,Very Good,F,SI1,63.1,57.0,5.72,5.75,3.62
193571,0.34,Very Good,D,SI1,62.9,55.0,4.45,4.49,2.81


In [14]:
cat_col=X.columns[X.dtypes=='object']
num_col=X.columns[X.dtypes!='object']

In [17]:
## another method
cat_col=X.select_dtypes(include='object').columns
num_col=X.select_dtypes(exclude='object').columns

In [18]:
cat_col

Index(['cut', 'color', 'clarity'], dtype='object')

In [19]:
num_col

Index(['carat', 'depth', 'table', 'x', 'y', 'z'], dtype='object')

In [20]:
# Define the custom ranking for each ordinal variable
cut_categories = ['Fair', 'Good', 'Very Good','Premium','Ideal']
color_categories = ['D', 'E', 'F', 'G', 'H', 'I', 'J']
clarity_categories = ['I1','SI2','SI1','VS2','VS1','VVS2','VVS1','IF']

In [21]:
from sklearn.preprocessing import StandardScaler
from sklearn.impute import SimpleImputer
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import OrdinalEncoder
from sklearn.compose import ColumnTransformer


In [22]:
num_pipeline=Pipeline(
    steps=[
        ('imputer',SimpleImputer(strategy='median')),
        ('scaler',StandardScaler())
    ]
)

cat_pipeline=Pipeline(
    steps=[
        ('imputer',SimpleImputer(strategy='most_frequent')),
        ('ordinalencoder',OrdinalEncoder(categories=[cut_categories,color_categories,clarity_categories])),
        ('scaler',StandardScaler())
    ]
)

mypipeline=ColumnTransformer([
    ('num_pipeline',num_pipeline,num_col),
    ('cat_pipeline',cat_pipeline,cat_col)
]
)

In [23]:
from sklearn.model_selection import train_test_split

In [24]:
x_train,x_test,y_train,y_test=train_test_split(X,Y,test_size=0.3,random_state=40)

In [25]:
x_train=pd.DataFrame(mypipeline.fit_transform(x_train),columns=mypipeline.get_feature_names_out())
x_test=pd.DataFrame(mypipeline.transform(x_test),columns=mypipeline.get_feature_names_out())

In [26]:
x_train

,num_pipeline__carat,num_pipeline__depth,num_pipeline__table,num_pipeline__x,num_pipeline__y,num_pipeline__z,cat_pipeline__cut,cat_pipeline__color,cat_pipeline__clarity
0,0.454106,-0.296545,-0.637842,0.636818,0.663672,0.622410,0.871502,-0.317026,-0.650314
1,3.027661,-1.589990,1.965484,2.494366,2.433180,2.182754,-0.133146,0.915044,-1.315348
2,0.540612,1.458846,-1.158507,0.672887,0.645523,0.811984,-1.137795,0.915044,-1.315348
3,0.562238,-2.513880,1.444819,0.817163,0.790714,0.549496,-1.137795,-0.317026,-1.315348
4,-0.843485,-0.019378,-0.637842,-0.851023,-0.888051,-0.865022,0.871502,1.531079,1.344786
...,...,...,...,...,...,...,...,...,...
135496,-0.000051,-0.019378,-0.637842,0.176940,0.200877,0.199512,0.871502,-0.933061,-0.650314
135497,-0.540714,-0.573712,1.444819,-0.409179,-0.443405,-0.471290,-0.133146,-0.317026,0.014719
135498,2.616757,1.828402,-0.637842,2.106625,2.061130,2.255668,-1.137795,0.915044,-1.315348
135499,0.454106,0.996901,-1.679173,0.564681,0.627374,0.680740,-2.142443,0.299009,0.679752


In [27]:
x_test

,num_pipeline__carat,num_pipeline__depth,num_pipeline__table,num_pipeline__x,num_pipeline__y,num_pipeline__z,cat_pipeline__cut,cat_pipeline__color,cat_pipeline__clarity
0,3.330432,0.165400,-0.637842,2.566503,2.532999,2.518156,0.871502,0.915044,-1.315348
1,-0.367702,0.165400,-0.117177,-0.228835,-0.207471,-0.194219,0.871502,-0.317026,-0.650314
2,0.865009,-0.204156,-1.158507,0.952421,0.999425,0.957811,0.871502,0.915044,-0.650314
3,0.475732,0.350178,-0.117177,0.582715,0.600151,0.636992,0.871502,1.531079,0.014719
4,-1.038124,0.534956,-1.158507,-1.193677,-1.232878,-1.171258,0.871502,0.299009,2.674852
...,...,...,...,...,...,...,...,...,...
58067,-0.497461,0.350178,0.403488,-0.382127,-0.443405,-0.383794,-0.133146,-0.933061,-1.315348
58068,-1.038124,0.904512,-0.117177,-1.265815,-1.296399,-1.215006,-0.133146,0.915044,-0.650314
58069,-0.713726,-0.850879,-0.637842,-0.643627,-0.624893,-0.690030,0.871502,-0.933061,0.679752
58070,1.124528,0.812123,0.403488,1.123748,1.162764,1.220299,-0.133146,0.915044,-1.315348


In [32]:
from sklearn.linear_model import LinearRegression,Ridge,Lasso,ElasticNet
from sklearn.metrics import r2_score,mean_absolute_error,mean_squared_error

In [31]:
def fun1(y_test,y_pred):
    r_sq=r2_score(y_test,y_pred)
    mse=mean_squared_error(y_test,y_pred)
    mae=mean_absolute_error(y_test,y_pred)
    rmse=np.sqrt(mse)
    return rmse,mae,r_sq


    
    

In [33]:
models={
    'linearregression':LinearRegression(),
    'ridge':Ridge(),
    'lasso':Lasso(),
    'elasticnet':ElasticNet()
}

In [39]:
for i in range(len(list(models))):
    model=list(models.values())[i]
    model.fit(x_train,y_train)
    y_pred=model.predict(x_test)
    rmse,mae,r_sq=fun1(y_test,y_pred)
    model_name=list(models.keys())[i]
    print("Model Trainer Performance")
    print(model_name)
    print("MAE",mae)
    print("RMSE",rmse)
    print("R_SQUARE",r_sq*100)
    print('*'*40)
    print('\n')
    
    

Model Trainer Performance
linearregression
MAE 674.6638217377892
RMSE 1017.1722564866152
R_SQUARE 93.63431403063537
****************************************


Model Trainer Performance
ridge
MAE 674.6868170797281
RMSE 1017.1731135131638
R_SQUARE 93.6343033037125
****************************************


Model Trainer Performance
lasso
MAE 675.3910275839803
RMSE 1015.6056722113335
R_SQUARE 93.65390698366912
****************************************


Model Trainer Performance
elasticnet
MAE 1065.5716610505156
RMSE 1536.886461343751
R_SQUARE 85.46751887552743
****************************************


